In [2]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import models
import paths
import shutil
from PIL import Image
import math
from os.path import join
from os import listdir
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


ImportError: No module named models

In [ ]:
desired_fold = 0

In [1]:
df = pd.read_json(paths.TRAIN_JSON)
listing_id_interest = df[['listing_id', 'interest_level']]

NameError: name 'pd' is not defined

In [4]:
# only use one k fold since we're using a cnn
trn_indices = []
val_indices = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)

for dev_index, val_index in kf.split(range(len(listing_id_interest))):
    trn_indices = dev_index
    val_indices = val_index
    if count == desired_fold:
        break
    count += 1
print len(trn_indices)
print len(val_indices)

39481
9871


In [10]:
ids = listing_id_interest['listing_id']
ids = np.array(ids)
val_ids = ids[val_indices]

In [12]:
print val_ids[0:19]

[7120132 6912559 6832226 6847390 6859368 6907631 7127110 6866313 6878074
 6890772 6940305 6846405 6819930 6864916 6883194 6914141 6885991 6822355
 6924176]


In [15]:
# creating val set
categories = ['low', 'medium', 'high']
for folder in categories:
    for img in listdir(join(paths.TRN_IMAGES, folder)):
        listing_id = int(img[:img.index('_')])
        if listing_id in val_ids:
            shutil.move(join(paths.TRN_IMAGES,folder,img), 
                        join(paths.VAL_IMAGES,folder,img))